### Homework

* 1 - Bajar el CSV de Oferta Gastronómica desde Buenos Aires Data (https://data.buenosaires.gob.ar/dataset/). Idealmente hacer esto con Python.

In [1]:
import pymysql
import urllib.request
from Pass_SQL import contraseña

In [2]:
archivo = urllib.request.urlopen('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv')

* 2 - Crear una tabla con los siguientes campos: id_local, nombre, categoria, direccion, barrio, comuna, para posteriormente poblarla con los datos bajados, utilzando el conector desde el script de Python.

In [3]:
# Hay campos en el archivo que tienen comillas, y en el medio una coma que debemos dejar. Esto nos va a dificultar la separación de campos
next(archivo) #Salteamos el header
lista_locales = [] #Fabricamos la lista vacía
for linea in archivo:
    linea = linea.decode().replace(',','|') #Leemos cada línea con decode, y reemplazamos , por |, porque algunas las vamos a necesitar luego
    while linea.find('"') != -1: #Buscamos las comillas, que son los campos que deben quedar con ,. Si da -1 es porque no hay
        pos1 = linea.find('"') #Buscamos la posición de la comilla
        pos2 = linea.find('"', pos1+1) #Buscamos la posición de la segunda comilla, para obtener el substring
        linea = linea[:pos1] + linea[pos1:pos2+1].replace('|',',').replace('"', '') + linea[pos2+1:]
        '''En esa linea se hacen varias cosas, se toma el string previo a las comillas, luego se agrega el substring que estaba entre comillas
        quitandole el | y devolviendole la , y también se quitan las comillas, y se agrega el resto del string.
        Esto se va a repetir si se vuelven a encontrar comillas, por el while'''
    lista_locales.append(tuple(linea.strip().split('|'))) #Se agregan los campos limpios como una tupla a la lista, obteniendo lista de tuplas

In [31]:
#Otra opción, que viene mejor parseada y nos ahorra trabajo
import requests, io, csv
documento = requests.get('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv')
documento.encoding = 'utf-8'
documento = csv.reader(io.StringIO(documento.text))
next(documento)
lista_locales = [tuple(linea) for linea in documento]

In [32]:
conexion = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = contraseña(),
    port = 3306,
    database = 'Oferta_gastronomica'
)
cursor = conexion.cursor()

En esta tabla "locales" metimos solo las columnas pedidas, como string

In [33]:
lista_corta = []
for elem in lista_locales:
    lista_corta.append(((elem[3],elem[4],elem[13],elem[14],elem[15][7:])))

In [14]:
cursor.executemany('''INSERT INTO locales (nombre,categoria,direccion,barrio,comuna)
                      VALUES (%s, %s, %s, %s, %s)''', lista_corta)
conexion.commit()

* 3 - A partir de tener los datos disponibles, responder a las siguientes preguntas:

  * a) ¿Cuál es el barrio con mayor cantidad de Pubs?
  * b) Obtener la cantidad de locales por categoría
  * c) Obtener la cantidad de restaurantes por comuna

In [35]:
# A ¿Cuál es el barrio con mayor cantidad de Pubs?
cursor.execute("""SELECT barrio, categoria, count(*) as Cantidad
                  FROM locales
                  GROUP BY 1, 2
                  HAVING categoria = 'PUB'
                  ORDER BY Cantidad DESC;""")
cursor.fetchmany(1)

(('Recoleta', 'PUB', 8),)

In [16]:
# B Obtener la cantidad de locales por categoría
cursor.execute("""SELECT categoria, count(*) as Cantidad_locales
                  FROM locales
                  GROUP BY 1;""")
cursor.fetchall()

(('RESTAURANTE', 1990),
 ('VINERIA', 22),
 ('CAFE', 334),
 ('BAR', 247),
 ('', 116),
 ('PUB', 53),
 ('CONFITERIA', 55),
 ('SANDWICHERIA', 4),
 ('DELIVERY & TAKE AWAY', 2))

In [36]:
# C) Obtener la cantidad de restaurantes por comuna
cursor.execute('''SELECT comuna, count(*) as Cantidad_restaurantes
                  FROM locales
                  WHERE categoria like '%RESTAURANTE%'
                  GROUP BY 1
                  ORDER BY 2 DESC;''')
cursor.fetchall()

((1, 609),
 (14, 366),
 (2, 227),
 (3, 145),
 (13, 108),
 (4, 95),
 (6, 87),
 (7, 64),
 (15, 57),
 (5, 56),
 (9, 52),
 (10, 43),
 (12, 42),
 (11, 33),
 (8, 6))

In [37]:
conexion.close()